# RSPt to Quanty
Notebook for analyzing RSPt output and translating it to Quanty. 

#### Recommended usage: Copy notebook to (or close to) simulation folder and tailor it to the system of study. 

Construct non-interacting Hamiltonian in spherical harmonics basis.
If RSPt cluster of study uses the cubic harmonic basis, an explicit rotation is done in the end of the notebook to spherical harmonic basis.
Otherwise a basis which numerically diagonalizes the local Hamiltonian is assumed.
In the end of the notebook a rotation from this diagonal basis to the spherical harmonic basis is done, by using the eigenvectors of the local Hamiltonian in the spherical harmonic basis. 

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
from math import pi
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

from py4rspt import rsptt   
from py4rspt import quantyt   
from py4rspt import unitarytransform   
from py4rspt.constants import eV
# Get the newest version
reload(quantyt)  
reload(rsptt)  
reload(unitarytransform)  

### Input parameters to adjust according to system of study 

In [ ]:
# Simulation folder to read information from. 
# Update path to desired directory
foldername = ('path/to/sim/folder/')
# Correlated orbitals to study, e.g.:
# '0102010100' or 
# '0102010103' or
# '0102010103-obs'
basis_tag = '0102010100'
# User defined local basis keyword. 
# If not used any user defined local basis by 
# providing a projection file, use an empty string: ''
irr_flag = 'Irr1'
# If to analyze spin-polarized calculations 
spinpol = True
# If to analyze spin averaged calculations. 
# This means spin-polarization, if any, is only from 
# the hybridization and the self-energy
spinavg = True
# Select bath energies. 
eb = np.array([[-4.],[-4.],[-4.],[-4.],[-4.],
               [-4.],[-4.],[-4.],[-4.],[-4.]])
eim = 0.005*eV # distance above real axis
# Energy mesh
w = np.linspace(-10,10,4500)
xlim = [-9,4]  # energy plot range
# Method of choice for calculating on-site energies.
# Three possibilities:
# 0 - Considers non-interacting PDOS and neglects 
#     off-diagonal elements of RSPt's hybridization 
#     function. 
# 1 - Considers non-interacting PDOS and includes 
#     off-diagonal elements of RSPt's hybridization 
#     function.
# 2 - Considers interacting PDOS and includes 
#     off-diagonal elements of RSPt's 
#     hybridization function.
e_method = 0
# If off-diagonal hybridization elements are 
# available in files
off_diag_hyb = True
# If self-energy is available in files
self_energy = True
# Energy intervall where to search for solution 
# of adjusted on-site energy
bounds = (-3,0.5)
# Energy window in which the center of gravity 
# of non-interacting PDOS is calculated. 
# Choose range by inspecting RSPt's 
# non-interacting PDOS.
wmin0 = -2
wmax0 = 2
# Energy window in which the center of gravity 
# of interacting PDOS is calculated. 
# Choose range by inspecting RSPt's 
# interacting PDOS.
wmin = -8
wmax = 3
verbose_fig = True
verbose_text = True

#### Help variables

In [ ]:
# If the diagonal Hamiltonian is due to use of cubic harmonics
cubic_basis = (basis_tag[-2:] == '03' or basis_tag[-6:] == '03-obs')
# Number of bath orbitals per correlated orbitals
nb = np.shape(eb)[1]     
# RSPt atomic type of interest
if basis_tag[0] == '0':
    at = int(basis_tag[1])
else:
    at = int(basis_tag[:2])  
# Number of orbitals, 5 for d-system, 7 for f-system
norb = 2*int(basis_tag[3])+1    
# Number of non-equivalent correlated spin-orbitals
nc = 2*norb if spinpol else norb  
# Number of non-equivalent on-site energies
no = 2*norb if spinpol and not spinavg else norb   
# Files for hybridization function
file_imag_hyb = foldername + 'imag-hyb-' + basis_tag + '.dat'
file_real_hyb = foldername + 'real-hyb-' + basis_tag + '.dat'
if off_diag_hyb:
    file_real_off_hyb = (foldername + 'real-offdiag-hyb-' 
                         + basis_tag + '.dat')
    file_imag_off_hyb = (foldername + 'imag-offdiag-hyb-' 
                         + basis_tag + '.dat')
if self_energy:
    # File for interacting PDOS
    file_pdos = foldername + 'pdos-' + basis_tag + '.dat'
    file_real_sig = foldername + 'real-sig-realaxis-' + basis_tag + '.dat'
    file_imag_sig = foldername + 'imag-sig-realaxis-' + basis_tag + '.dat'
    file_real_sig_offd = (foldername 
                          + 'real-offdiag-sig-realaxis-' 
                          + basis_tag + '.dat')
    file_imag_sig_offd = (foldername 
                          + 'imag-offdiag-sig-realaxis-' 
                          + basis_tag + '.dat')
assert np.shape(eb)[0] == nc

#### Plot RSPt's hybridization function
Plot only diagonal elements, even if off-diagonal elements exist.

In [ ]:
if verbose_fig:
    x = np.loadtxt(file_imag_hyb)
    wp = x[:,0]*eV
    mask = np.logical_and(xlim[0]<wp,wp<xlim[1])
    wp = wp[mask]
    # total hybridization
    plt.figure()
    hyb_tot = -1/pi*x[mask,1]*eV
    plt.plot(wp,hyb_tot) 
    plt.xlabel('$\omega$   (eV)')
    plt.ylabel(r'$\frac{-1}{\pi}$ Im $\Delta(\omega)$')
    plt.xlim(xlim)
    plt.grid(color='0.9')
    plt.show()
    if spinpol:
        # Spin up and down
        plt.figure()
        hyb_dn = -1/pi*x[mask,2]*eV
        plt.plot(wp,-hyb_dn,c='C0') 
        hyb_up = -1/pi*x[mask,3]*eV
        plt.plot(wp,hyb_up,c='C0') 
        plt.xlabel('$\omega$   (eV)')
        plt.ylabel(r'$\frac{-1}{\pi}$ Im $\Delta(\omega)$')
        plt.xlim(xlim)
        plt.grid(color='0.9')
        plt.show()
        # Orbital resolved
        fig = plt.figure()
        for i in range(norb):
            hyb_dn = -1/pi*x[mask,4+i]*eV 
            hyb_up = -1/pi*x[mask,4+norb+i]*eV  
            plt.plot(wp,hyb_dn+hyb_up,c='C'+str(i),label=str(i)) 
        plt.ylabel(r'$\frac{-1}{\pi}$ Im $\Delta(\omega)$')
        plt.grid(color='0.9')
        plt.legend()
        plt.xlabel('$\omega$   (eV)')
        plt.xlim(xlim)
        plt.tight_layout()
        plt.show()
        # Orbital and spin resolved
        fig = plt.figure()
        for i in range(norb):
            hyb_dn = -1/pi*x[mask,4+i]*eV
            # Plot down spin with minus sign
            plt.plot(wp,-hyb_dn,c='C'+str(i),label=str(i))   
            hyb_up = -1/pi*x[mask,4+norb+i]*eV
            plt.plot(wp,hyb_up,c='C'+str(i)) 
        plt.ylabel(r'$\frac{-1}{\pi}$ Im $\Delta(\omega)$')
        plt.grid(color='0.9')
        plt.legend()
        plt.xlabel('$\omega$   (eV)')
        plt.xlim(xlim)
        plt.tight_layout()
        plt.show()
        # Orbital and spin resolved
        fig, axes = plt.subplots(nrows=norb,sharex=True,sharey=True)
        for i,ax in enumerate(axes):
            hyb_dn = -1/pi*x[mask,4+i]*eV
            # Plot down spin with thinner line
            ax.plot(wp,hyb_dn,lw=0.7,c='C'+str(i))  
            hyb_up = -1/pi*x[mask,4+norb+i]*eV
            ax.plot(wp,hyb_up,c='C'+str(i),label=str(i)) 
            ax.grid(color='0.9')
            ax.legend()
        axes[-1].set_ylabel(r'$\frac{-1}{\pi}$ Im $\Delta(\omega)$')
        axes[-1].set_xlabel('$\omega$   (eV)')
        axes[-1].set_xlim(xlim)
        plt.tight_layout()
        plt.show()
    else:
        # Orbital resolved
        fig = plt.figure()
        for i in range(nc):
            hyb_i = -1/pi*x[mask,4+i]*eV 
            plt.plot(wp,hyb_i,label=str(i)) 
        plt.ylabel(r'$\frac{-1}{\pi}$ Im $\Delta(\omega)$')
        plt.grid(color='0.9')
        plt.legend()
        plt.xlabel('$\omega$   (eV)')
        plt.xlim(xlim)
        plt.tight_layout()
        plt.show()        

#### Calculate hybridization hoppings

In [ ]:
# Load the hybridization function
hyb_re = np.loadtxt(file_real_hyb)
hyb_im = np.loadtxt(file_imag_hyb)
wd = eV*hyb_im[:,0]
hybridization_re = np.zeros((nc,len(wd)))
hybridization_im = np.zeros((nc,len(wd)))
for i in range(nc):
    hybridization_re[i,:] = hyb_re[:,4+i]*eV
    hybridization_im[i,:] = hyb_im[:,4+i]*eV
# Complex-valued hybridization function
hybridization = hybridization_re + hybridization_im*1j
print 'Calculate hopping elements'
vb,wborder = rsptt.get_v_from_eb(wd,hybridization_im,eb,
                                 accept1=0.2,accept2=0.5)
hyb = rsptt.hyb_d(w+1j*eim,eb,vb)
print 'eb:'
print eb
print 'wborder:'
print wborder
print 'vb:'
print vb
s = '{:6.3f},'*nb
print 'hoppings (eV):'
for i in range(nc):
    print str(i) + ': V_b =' + s.format(*vb[i]) 

#### Plot discretized hybridization function

In [ ]:
if verbose_fig:
    fig, axarr = plt.subplots(nc,figsize=(7,8),sharex=True,
                              sharey=True)
    # Loop over non-equivalent correlated spin-orbitals
    for i,ax in enumerate(axarr):
        color=iter(plt.cm.rainbow(np.linspace(0,1,nb)))    
        ax_v = ax.twinx()
        ax_v.set_ylabel('$v_{b}$  (eV)', color='r')
        ax_v.tick_params('y', colors='r')
        # Loop over bath states
        for ebath,v,wb,c in zip(eb[i],vb[i],wborder[i],color):
            ax_v.plot([ebath,ebath],[0,v],c='r')
            ax_v.plot([wb[0],wb[0]],[0,v],'--',c=c)
            ax_v.plot([wb[1],wb[1]],[0,v],'--',c=c)
        # Plot discretized hybridization function
        ax.plot(w,-1/pi*np.imag(hyb[i,:]),c='0.8',label='discrete')
        v_max_plot = 1.07*np.max(vb[i])
        ax_v.set_ylim([0,v_max_plot])
        ax.grid(color='0.92')
    axarr[int(nc/2)].set_ylabel(r'-$\frac{1}{\pi}\mathrm{Im}'
                                  '\Delta(\omega)$      (eV)')
    # Plot continues hybridization function
    for i,ax in enumerate(axarr):
        hyb_max_plot = 1.2*np.max(-1/pi*hybridization_im[i])
        ax.set_ylim([0,hyb_max_plot])
        ax.plot(wd,-1/pi*hybridization_im[i],'-k',label='RSPt')
    axarr[-1].set_xlabel('E   (eV)')
    axarr[0].set_xlim(xlim)
    axarr[0].legend(loc='best')
    plt.subplots_adjust(left=0.17,bottom=0.10,right=0.83,
                        top=0.98,hspace=0.0,wspace=0)
    plt.show()

    if spinpol:
        fig, axarr = plt.subplots(norb,figsize=(6,7),sharex=True)
        # Loop over axes
        for i,ax in enumerate(axarr):
            ax_v = ax.twinx()
            ax_v.set_ylabel('$v_{b}$  (eV)', color='r')
            ax_v.tick_params('y', colors='r')

            for j,s in zip([i,norb+i],[-1,1]):
                color=iter(plt.cm.rainbow(np.linspace(0,1,nb)))    
                # Loop over bath states
                for ebath,v,wb,c in zip(eb[j],vb[j],
                                        wborder[j],color):
                    ax_v.plot([ebath,ebath],[0,s*v],c='r')
                    ax_v.plot([wb[0],wb[0]],[0,s*v],'--',c=c)
                    ax_v.plot([wb[1],wb[1]],[0,s*v],'--',c=c)
            # Plot discretized hybridization function
            ax.plot(w,--1/pi*np.imag(hyb[i,:]),
                    c='0.8',label='discrete')
            ax.plot(w,-1/pi*np.imag(hyb[norb+i,:]),c='0.8')
            v_max_dn = 1.1*np.max(vb[i])
            v_max_up = 1.1*np.max(vb[norb+i])
            v_max = max(v_max_dn,v_max_up)
            ax_v.set_ylim([-v_max,v_max])
            ax.grid(color='0.92')
        axarr[2].set_ylabel(r'-$\frac{1}{\pi}\mathrm{Im}'
                            '\Delta(\omega)$      (eV)')
        # Plot RSPt hybridization function
        for i,ax in enumerate(axarr):
            hyb_max_dn = 1.4*np.max(-1/pi*hybridization_im[i])
            hyb_max_up = 1.4*np.max(-1/pi*hybridization_im[norb+i])
            hyb_max = max(hyb_max_dn,hyb_max_up)
            ax.set_ylim([-hyb_max,hyb_max])
            ax.plot(wd,--1/pi*hybridization_im[i],'-r',label='RSPt')
            ax.plot(wd,-1/pi*hybridization_im[norb+i],'-r')
        axarr[-1].set_xlabel('E   (eV)')
        axarr[0].set_xlim(xlim)
        axarr[0].legend(loc='best')
        plt.subplots_adjust(left=0.17,bottom=0.10,right=0.83,
                            top=0.98,hspace=0.0,wspace=0)
        plt.show()
        # New figure
        fig, axarr = plt.subplots(norb,figsize=(6,7),sharex=True)
        # Loop over axes
        for i,ax in enumerate(axarr):
            # Down spin has negative sign
            ax.plot(w,--1/pi*np.imag(hyb[i,:]),c='0.8')
            ax.plot(wd,--1/pi*hybridization_im[i],'-r')
            # Up spin has positive sign
            ax.plot(w,-1/pi*np.imag(hyb[norb+i,:]),
                    c='0.8',label='discrete')
            ax.plot(wd,-1/pi*hybridization_im[norb+i],
                    '-r',label='RSPt')
            y_max_dn = 1.4*np.min(-1/pi*hybridization_im[i])
            y_max_up = 1.4*np.max(-1/pi*hybridization_im[norb+i])
            y_max = max(y_max_dn,y_max_up)
            ax.text(0, 2, i, fontsize=8, 
                    fontweight='normal',
                    bbox={'facecolor':'white', 'alpha':0.9, 
                          'pad':2})
            ax.set_ylim([-y_max,y_max])
        axarr[0].set_ylabel(r'-$\frac{1}{\pi}\mathrm{Im}'
                            '\Delta(\omega)$  (eV)')
        axarr[-1].set_xlabel('E   (eV)')
        axarr[0].set_xlim(xlim)
        axarr[0].legend(loc=1)
        plt.subplots_adjust(left=0.16,bottom=0.16,right=0.99,
                            top=0.99,hspace=0,wspace=0)
        plt.show()

#### Calculate RSPt's non-interacting PDOS

An approximation of the non-interacting PDOS is given by $-\frac{1}{\pi} \mathrm{Im}g_{0,d}^{\mathrm{rspt}}(\omega)$,

where $g_{0,d}^{\mathrm{rspt}}(\omega) = (\omega-\epsilon_{d,d}^\mathrm{rspt}-\Delta_{d,d}^\mathrm{rspt}(\omega))^{-1}$.

Also construct $g_{0,d}(\omega) = (\omega-\epsilon_{d,d}-\Delta_{d,d}(\omega))^{-1}$ with $\epsilon_{d,d} = \epsilon_{d,d}^\mathrm{rspt}$ and $\Delta_{d,d}(\omega)$ being the discretized hybridization function.

The approximation is that off-diagonal elements of the hybridization function are neglected.
This allow us to treat each spin-orbital seperatly.

In [ ]:
def get_pdos0_eig_weight(e,b,v,w,eim,pdos_method='0'):
    '''return non-interacting PDOS for correlated orbitals.
    
    Eigenvalues and weights of non-interacting Hamiltonian 
    is also returned. 
    
    Input arguments:
    e - on-site energies of correlated orbitals
    b - on-site energies of bath orbitals
    v - hopping elements
    w - real energy mesh
    eim - distance above real axis
    pdos_method - '0' or '1'
    '''
    if pdos_method == '0':
        # Calculate the hamiltonian
        ham = rsptt.h_d0(e,b,v)
        # Calculate the PDOS (using the Hamiltonian)
        pdos = rsptt.pdos_d0_1(w,eim,ham)
        # Calculate eigenvalues and weights of Hamiltonian
        eig,weight = rsptt.eig_and_weight1(ham)
    elif pdos_method == '1':
        # Calculate the hybridization function
        hyb = rsptt.hyb_d(w+1j*eim,b,v)
        # Calulate the PDOS (using the hybridization function) 
        pdos = rsptt.pdos(w,eim,e,hyb)
        eig,weight = None,None
        print 'Warning: eigenvalues and weights are not calculated'
    return pdos,eig,weight

# Extract the impurity energies from the local Hamiltonian 
# and the chemical potential
hs,labels = rsptt.parse_matrices(foldername + 'out')   
mu = rsptt.mu(foldername + 'out')
for h,label in zip(hs,labels):
    # Select Hamiltonian from correct cluster
    if label == basis_tag:
        print label
        e_rspt = eV*np.real(h.diagonal()-mu)
e_rspt = e_rspt[:no]  # select non-equivalent on-site energies
print('e_rspt:',e_rspt)
# Construct the non-interacting PDOS, in three different ways.
# 1) Use RSPt hybridization function.
# 2) Use discretized hybridization function.
# 3) Use discretized hybridization function, 
#    but calculated in another way.
if no == nc:    
    p0d_rspt = rsptt.pdos(wd,eim,e_rspt,hybridization)
    p0d_alg1 = rsptt.pdos(w,eim,e_rspt,hyb)
    p0d_alg2,eig,weight = get_pdos0_eig_weight(e_rspt,eb,vb,w,eim)
else:
    p0d_rspt = rsptt.pdos(wd,eim,2*list(e_rspt),hybridization)
    p0d_alg1 = rsptt.pdos(w,eim,2*list(e_rspt),hyb)
    p0d_alg2,eig,weight = get_pdos0_eig_weight(2*list(e_rspt),
                                               eb,vb,w,eim)
assert np.all(np.abs(p0d_alg1 - p0d_alg2) < 1e-13)
# Interpolate RSPt's non-interacting PDOS onto another mesh
p0d_rspti = np.zeros_like(p0d_alg1)
for i in range(nc):
    p0d_rspti[i,:] =  interp1d(wd,p0d_rspt[i,:],
                               bounds_error=False,fill_value=0)(w)

#### Plot non-interacting PDOS

In [ ]:
if verbose_fig:
    # Plot non-interacting PDOS
    plt.figure()
    for i in range(nc):
        plt.plot(w,p0d_rspti[i,:],c='C'+str(i),label=str(i))
        plt.plot(w,p0d_alg1[i,:],'--',c='C'+str(i))
    plt.legend()
    plt.ylabel('PDOS$_0$')
    plt.xlim(xlim)
    plt.show()
    if spinpol:
        # Plot non-interacting PDOS
        # Down spin with negative sign
        plt.figure()
        for i in range(nc/2):
            # Down spin
            plt.plot(w,-p0d_rspti[i,:],c='C'+str(i),label=str(i))
            plt.plot(w,-p0d_alg1[i,:],'--',c='C'+str(i))
            # Up spin
            plt.plot(w,p0d_rspti[nc/2+i,:],c='C'+str(i))
            plt.plot(w,p0d_alg1[nc/2+i,:],'--',c='C'+str(i))
        plt.legend()
        plt.ylabel('PDOS$_0$')
        plt.xlim(xlim)
        plt.show()

#### Adjust $\epsilon^{(0,d)}$ so that $g_{0,d}(\omega) \approx g_{0,d}^{\mathrm{rspt}}(\omega)$ 
Consider the center of gravity in an energy region where most states are located.

In [ ]:
def err(e,eb,vb,w,eim,pdos_default,wmin,wmax):
    '''returns deviation of center of gravity between default PDOS 
    and the constructed non-intercating PDOS.
    '''
    h = rsptt.h_d0(e,eb,vb)
    pdos0 = rsptt.pdos_d0_1(w,eim,h)
    return np.abs(rsptt.averageE(w,pdos0,wmin,wmax)
                  -rsptt.averageE(w,pdos_default,wmin,wmax))

def err2(e,ebs,vbs,w,eim,pdos_defaults,wmin,wmax):
    '''returns deviation of center of gravity between default PDOS 
    and the constructed non-intercating PDOS.
    '''
    s = 0
    for eb,vb,pdos_default in zip(ebs,vbs,pdos_defaults):
        h = rsptt.h_d0(e,eb,vb)
        pdos0 = rsptt.pdos_d0_1(w,eim,h)
        s += np.abs(rsptt.averageE(w,pdos0,wmin,wmax)
                    -rsptt.averageE(w,pdos_default,wmin,wmax))**2
    return s

wmin0s = [wmin0]*no
wmax0s = [wmax0]*no
e0d = np.zeros_like(e_rspt)
# Optimize on-site energy by fitting to non-interacting PDOS, 
# while keeping bath parameters fix
if nc == no:
    for i in range(no):
        res = minimize_scalar(err, bounds=bounds, 
                              args=(eb[i],vb[i],w,eim,p0d_rspti[i],
                                    wmin0s[i],wmax0s[i]), 
                              method='bounded')
        e0d[i] = res['x']
        if verbose_text:
            print 'err(e0d) =',res.fun
            print 'err(e_rspt) =',err(e_rspt[i],eb[i],vb[i],w,eim,
                                      p0d_rspti[i],wmin0s[i],
                                      wmax0s[i])
            print
    p0d = rsptt.pdos(w,eim,e0d,hyb)
else:
    for i in range(no):
        res = minimize_scalar(err2, bounds=bounds, 
                              args=([eb[i],eb[i+no]],
                                    [vb[i],vb[i+no]],
                                    w,eim,[p0d_rspti[i],
                                           p0d_rspti[i+no]],
                                    wmin0s[i],wmax0s[i]), 
                              method='bounded')
        e0d[i] = res['x']
        if verbose_text:
            print 'err(e0d) =',res.fun
            print 'err(e_rspt) =',err2(e_rspt[i],[eb[i],eb[i+no]],
                                       [vb[i],vb[i+no]],w,eim,
                                       [p0d_rspti[i],
                                        p0d_rspti[i+no]],
                                       wmin0s[i],wmax0s[i])
            print
    p0d = rsptt.pdos(w,eim,2*list(e0d),hyb)
print 'e_rspt:',e_rspt,'eV'
print 'e_0d:',e0d,'eV'
print
# Check how well the fitting worked
if verbose_text:
    for i in range(nc):
        if i < no:
            print '<w>_p0d_rspt =',rsptt.averageE(w,p0d_rspti[i],
                                                  wmin0s[i],
                                                  wmax0s[i])
            print '<w>_p0d =',rsptt.averageE(w,p0d[i],wmin0s[i],
                                             wmax0s[i])
        else:
            print '<w>_p0d_rspt =',rsptt.averageE(w,p0d_rspti[i],
                                 wmin0s[i-norb],wmax0s[i-norb])
            print '<w>_p0d =',rsptt.averageE(w,p0d[i],
                                 wmin0s[i-norb],wmax0s[i-norb])
        print

#### Plot non-interacting PDOS
Use $\epsilon_{0,d}$ and $\epsilon_\mathrm{rspt}$

In [ ]:
if verbose_fig:
    # Trace
    plt.figure()
    plt.plot(w,np.sum(p0d_rspti,axis=0),label='p0d_rspt')
    plt.plot(w,np.sum(p0d,axis=0),label='p0d')
    plt.legend()
    plt.ylabel('PDOS$_0$')
    plt.xlim(xlim)
    plt.show()
    if spinpol:
        # Down spin with negative sign
        plt.figure()
        for i in range(norb):
            # Down spin
            plt.plot(w,-p0d_rspti[i,:],c='C'+str(i),label=str(i))
            plt.plot(w,-p0d[i,:],'--',c='C'+str(i))
            # Up spin
            plt.plot(w,p0d_rspti[norb+i,:],c='C'+str(i))
            plt.plot(w,p0d[norb+i,:],'--',c='C'+str(i))
        plt.legend()
        plt.ylabel('PDOS$_0$')
        plt.xlim(xlim)
        plt.show()

    # Compare with e_rspt
    fig,axes = plt.subplots(nc,figsize=(6,10),sharex=True)
    # Plot calculated PDOS (using e_rspt)
    for ax,y in zip(axes,p0d_alg1):
        ax.plot(w,y,'-b',label='$\epsilon_\mathrm{rspt}$')
    # Plot calculated PDOS (using e_0d)
    for ax,y in zip(axes,p0d):
        ax.plot(w,y,'-g',label='$\epsilon$')
    # Plot original non-interacting PDOS
    for t,ax in enumerate(axes):
        ax.plot(w,p0d_rspti[t],'-r',label='RSPt')
        ax.set_ylabel(str(t))
    # Figure design
    axes[-1].set_xlabel('$\omega$  (eV)')
    axes[0].legend(loc=2)
    axes[0].set_xlim(xlim)
    for i,ax in enumerate(axes):
        ax.grid()
    plt.subplots_adjust(left=0.15,bottom=0.11,right=0.99,
                        top=0.98,hspace=0,wspace=0)
    plt.show()
    
    if spinpol:
        fig,axes = plt.subplots(norb,figsize=(6,6),sharex=True)
        # Plot calculated PDOS (using e_rspt)
        for i,ax in enumerate(axes):
            ax.plot(w,-p0d_alg1[i,:],
                    '-b',label='$\epsilon_\mathrm{rspt}$')
            ax.plot(w,p0d_alg1[norb+i,:],'-b')
        # Plot calculated PDOS (using e)
        for i,ax in enumerate(axes):
            ax.plot(w,-p0d[i,:],'-g',label='$\epsilon$')
            ax.plot(w,p0d[norb+i,:],'-g')
        # Plot original PDOS
        for i,ax in enumerate(axes):
            ax.plot(w,-p0d_rspti[i],'-r',label='RSPt')
            ax.plot(w,p0d_rspti[norb+i],'-r')
            ax.set_ylabel(str(i))
        # Figure design
        axes[-1].set_xlabel('$\omega$  (eV)')
        axes[0].legend(loc=2)
        axes[0].set_xlim(xlim)
        plt.subplots_adjust(left=0.15,bottom=0.11,right=0.99,
                            top=0.98,hspace=0,wspace=0)
        plt.show()

#### Off-diagonal hybridization
Consider off-diagonal hybridization elements.

In [ ]:
if off_diag_hyb: 
    # Read off-diagonal hybridization functions
    re = eV*np.loadtxt(file_real_off_hyb)[:,1:]
    im = eV*np.loadtxt(file_imag_off_hyb)[:,1:]
    hybridization_matrix = np.zeros((nc,nc,len(wd)),
                                    dtype=np.complex)
    # Diagonal
    for i in range(nc):
        hybridization_matrix[i,i,:] = hybridization[i,:]
    # Off-diagonal
    n = 0
    for j in range(nc):
        if spinpol:
            if j<norb:
                irange = range(j)+range(j+1,norb)
            else:
                irange = range(norb,j)+range(j+1,nc)
        else:
            irange = range(j)+range(j+1,norb)        
        for i in irange:
            hybridization_matrix[i,j,:] = re[:,n]+im[:,n]*1j
            n += 1
    # Calculate RSPt non-interacting PDOS, 
    # using full-matrix RSPt hybridization function
    if nc == no:
        p0_rspt = np.diagonal(rsptt.pdos(
            wd,eim,e_rspt,hybridization_matrix)).T
    elif nc == 2*no:
        p0_rspt = np.diagonal(rsptt.pdos(
            wd,eim,2*list(e_rspt),hybridization_matrix)).T
    # Interpolate non-interacting RSPt PDOS onto another mesh
    p0_rspti = np.zeros_like(p0d)
    for i in range(nc):
        p0_rspti[i,:] =  interp1d(wd,p0_rspt[i,:],
                                  bounds_error=False,
                                  fill_value=0)(w)        

#### Plot off-diagonal hybridization elements

In [ ]:
if verbose_fig and off_diag_hyb:
    plt.figure()
    for i in range(nc):
        plt.plot(wd,-np.imag(
            hybridization_matrix[i,i,:]),'-k')
    for i in range(nc):
        for j in range(nc):
            if i != j:
                plt.plot(wd,-np.imag(
                    hybridization_matrix[i,j,:]),'-r')
    plt.plot([],[],'-k',label='diagonal')
    plt.plot([],[],'-r',label='off-diagonal')
    plt.legend()
    plt.xlim(xlim)
    plt.show()
    plt.figure()
    plt.plot(w,np.sum(p0d_rspti,axis=0),label='p0d_rspt')
    plt.plot(w,np.sum(p0d,axis=0),label='p0d')
    plt.plot(w,np.sum(p0_rspti,axis=0),label='p0_rspt')
    plt.legend()
    plt.ylabel('PDOS$_0$')
    plt.xlim(xlim)
    plt.show()

#### Adjust $\epsilon^{(0)}$ so that $g_{0}(\omega) \approx g_{0}^{\mathrm{rspt}}(\omega)$ 
where

$ g_{0}^{\mathrm{rspt}}(\omega) =  ((\omega+i\delta)\delta_{a,b}-\epsilon^\mathrm{rspt}_{a,a}-\Delta^\mathrm{rspt}_{a,b}(\omega))^{-1}$

and

$ g_{0}(\omega) =  ((\omega+i\delta)\delta_{a,b}-\epsilon_{a,a}^{(0)}-\Delta_{a,a}(\omega))^{-1}$.

Consider the center of gravity in an energy region where most states are located.

In [ ]:
if off_diag_hyb:
    e0 = np.zeros_like(e_rspt)
    if nc == no:
        for i in range(nc):
            res = minimize_scalar(err, bounds=bounds, 
                                  args=(eb[i],vb[i],w,eim,
                                        p0_rspti[i],
                                        wmin0s[i],wmax0s[i]), 
                                  method='bounded')
            e0[i] = res['x']
            if verbose_text:
                print 'err(e0) =',res.fun
                print 'err(e_rspt) =',err(e_rspt[i],eb[i],vb[i],
                                          w,eim,p0_rspti[i],
                                          wmin0s[i],wmax0s[i])
                print
        p0 = rsptt.pdos(w,eim,e0,hyb)
    else:
        for i in range(no):
            res = minimize_scalar(err2, bounds=bounds, 
                                  args=([eb[i],eb[i+no]],
                                        [vb[i],vb[i+no]],
                                        w,eim,
                                        [p0_rspti[i],
                                         p0_rspti[i+no]],
                                        wmin0s[i],wmax0s[i]), 
                                  method='bounded')
            e0[i] = res['x']
            if verbose_text:
                print 'err2(e0) =',res.fun
                print 'err2(e_rspt) =',err2(e_rspt[i],
                                            [eb[i],eb[i+no]],
                           [vb[i],vb[i+no]],w,eim,
                           [p0_rspti[i],p0_rspti[i+no]],
                           wmin0s[i],wmax0s[i])
                print
        p0 = rsptt.pdos(w,eim,2*list(e0),hyb)
    if verbose_text:
        print '<w>_p0d_rspt =',rsptt.averageE(
            w,np.sum(p0d_rspti,axis=0),wmin0s[0],wmax0s[0])
        print '<w>_p0d =',rsptt.averageE(w,np.sum(p0d,axis=0),
                             wmin0s[0],wmax0s[0])
        print '<w>_p0_rspt =',rsptt.averageE(
            w,np.sum(p0_rspti,axis=0),wmin0s[0],wmax0s[0])
        print '<w>_p0 =',rsptt.averageE(w,np.sum(p0,axis=0),
                                        wmin0s[0],wmax0s[0])
        print
    print 'e_rspt:',e_rspt,'eV'
    print 'e_0d:',e0d,'eV'
    print 'e_0:',e0,'eV'

#### Plot non-interacting PDOS, including off-diagonal hybridization elements

In [ ]:
if verbose_fig and off_diag_hyb:
    plt.figure()
    plt.plot(w,np.sum(p0d_rspti,axis=0),label='p0d_rspt')
    plt.plot(w,np.sum(p0d,axis=0),label='p0d')
    plt.plot(w,np.sum(p0_rspti,axis=0),label='p0_rspt')
    plt.plot(w,np.sum(p0,axis=0),label='p0')
    plt.legend()
    plt.ylabel('PDOS$_0$')
    plt.xlim(xlim)
    plt.show()
    if spinpol:
        fig,axes = plt.subplots(norb,figsize=(6,6),sharex=True)
        for i,ax in enumerate(axes):
            ax.plot(w,-p0d[i,:],'--g',label='$\epsilon_{0,d}$')
            ax.plot(w,p0d[norb+i,:],'--g')
            ax.plot(w,-p0[i,:],'-g',label='$\epsilon_{0}$')
            ax.plot(w,p0[norb+i,:],'-g')

            ax.plot(w,-p0d_rspti[i],'--r',label='p0d_RSPt')
            ax.plot(w,p0d_rspti[norb+i],'--r')
            ax.plot(w,-p0_rspti[i],'-r',label='p0_RSPt')
            ax.plot(w,p0_rspti[norb+i],'-r')

            ax.set_ylabel(str(i))
        # Figure design
        axes[-1].set_xlabel('$\omega$  (eV)')
        axes[0].legend(loc=2)
        axes[0].set_xlim(xlim)
        plt.subplots_adjust(left=0.15,bottom=0.11,
                            right=0.99,top=0.98,hspace=0,wspace=0)
        plt.show()

### Interacting PDOS
Compare interacting PDOS from RSPt with a discretized version.

Self-energy needed.

In [ ]:
if self_energy:
    # Load interacting PDOS from file
    x = np.loadtxt(file_pdos)
    p_rspt = np.zeros((nc,len(wd)))
    k = 7 if spinpol else 2
    for i in range(nc):
        p_rspt[i,:] = x[:,k+i]/eV
    # Interpolate interacting RSPt PDOS onto another mesh
    p_rspti = np.zeros_like(p0)
    for i in range(nc):
        p_rspti[i,:] =  interp1d(wd,p_rspt[i,:],bounds_error=False,
                                 fill_value=0)(w)
    
    # Construct self-energy
    sigma = np.zeros((nc,nc,len(wd)),dtype=np.complex)
    # Read diagonal part of the self-energy
    sigma_d = eV*(np.loadtxt(file_real_sig)[:,4:]
                + 1j*np.loadtxt(file_imag_sig)[:,4:]).T
    # Diagonal
    for i in range(nc):
        sigma[i,i,:] = sigma_d[i,:]
    # Read off-diagonal part of the self-energy 
    re = eV*np.loadtxt(file_real_sig_offd)[:,1:]
    im = eV*np.loadtxt(file_imag_sig_offd)[:,1:]
    # Off-diagonal
    n = 0
    for j in range(nc):
        if spinpol:
            if j<norb:
                irange = range(j)+range(j+1,norb)
            else:
                irange = range(norb,j)+range(j+1,nc)
        else:
            irange = range(j)+range(j+1,norb)        
        for i in irange:
            sigma[i,j,:] = re[:,n]+im[:,n]*1j
            n += 1    
    # Calculate RSPt PDOS in two ways:
    # - Using full-matrix RSPt hybridization function
    # - Using diagonal RSPt hybridization function 
    if off_diag_hyb:
        if nc == no:
            p_rspt_alg1 = np.diagonal(rsptt.pdos(
                wd,eim,e_rspt,hybridization_matrix,sigma)).T
            p_rspt_alg2 = np.diagonal(rsptt.pdos(
                wd,eim,e_rspt,hybridization,sigma)).T
        elif nc == 2*no:
            p_rspt_alg1 = np.diagonal(rsptt.pdos(
                wd,eim,2*list(e_rspt),hybridization_matrix,sigma)).T
            p_rspt_alg2 = np.diagonal(rsptt.pdos(
                wd,eim,2*list(e_rspt),hybridization,sigma)).T

#### Plot interacting PDOS

In [ ]:
if verbose_fig and self_energy:
    plt.figure()
    plt.plot(wd,np.sum(p_rspt,axis=0),label='RSPt')
    if off_diag_hyb:
        plt.plot(wd,np.sum(p_rspt_alg1,axis=0),
                 '--',label='RSPt, alg1')
    plt.plot(wd,np.sum(p_rspt_alg2,axis=0),label='RSPt, alg2')
    plt.legend()
    plt.ylabel('PDOS')
    plt.xlim(xlim)
    plt.show()

    if spinpol:
        plt.figure()
        plt.plot(wd,-np.sum(p_rspt[:norb,:],axis=0),
                 c='C0',label='RSPt')
        if off_diag_hyb:
            plt.plot(wd,-np.sum(p_rspt_alg1[:norb,:],axis=0),
                     '--',c='C1',label='RSPt, alg1')
        plt.plot(wd,-np.sum(p_rspt_alg2[:norb,:],axis=0),
                 c='C2',label='RSPt, alg2')
        plt.plot(wd,np.sum(p_rspt[norb:,:],axis=0),c='C0')
        if off_diag_hyb:
            plt.plot(wd,np.sum(p_rspt_alg1[norb:,:],axis=0),
                     '--',c='C1')
        plt.plot(wd,np.sum(p_rspt_alg2[norb:,:],axis=0),c='C2')
        plt.legend()
        plt.ylabel('PDOS')
        plt.xlim(xlim)
        plt.show()

        plt.figure()
        for i in range(norb):
            plt.plot(wd,-p_rspt[i,:],c='C'+str(i),label=str(i))
            plt.plot(wd,p_rspt[norb+i,:],c='C'+str(i))
        plt.legend()
        plt.ylabel('PDOS')
        plt.xlim(xlim)
        plt.show()

#### Adjust $\epsilon$ so that $g(\omega) \approx g^{\mathrm{rspt}}(\omega)$ 
where

$ g^{\mathrm{rspt}}(\omega) =  ((\omega+i\delta)\delta_{a,b}-\epsilon^\mathrm{rspt}_{a,a}-\Delta^\mathrm{rspt}_{a,b}(\omega)-\Sigma_{a,b})^{-1}$

and

$ g(\omega) =  ((\omega+i\delta)\delta_{a,b}-\epsilon_{a,a}^{(0)}-\Delta_{a,a}(\omega)-\Sigma_{a,b})^{-1}$.

Consider the center of gravity in an energy region where most states are located.

In [ ]:
def errv(e,avgErspt,w,eim,hyb,sig):
    '''Error function to be minimized.
    
    Used to find optimal impurity energies for the finite 
    impurity model.
    
    Input parameters:
    e - on-site energies. 1d
    avgErspt - average energy of interacting PDOS, within 
               certain energy window. 1d
    w - energy mesh. 1d
    eim - distance above real axis
    hyb - discrete hybridization function. 3d
    sig - RSPt dynamic self-energy. 3d 
    '''
    nc = len(avgErspt)
    # Calculate approximative PDOS
    if len(e) == nc:
        p = np.diagonal(rsptt.pdos(w,eim,e,hyb,sig))
    elif 2*len(e) == nc:
        p = np.diagonal(rsptt.pdos(w,eim,2*list(e),hyb,sig))     
    # Sum of deviations of average energies
    s = 0
    for i in range(nc):
        s += abs(rsptt.averageE(w,p[:,i])-avgErspt[i])**2
    return s

if self_energy:
    mask = np.logical_and(wmin<w,w<wmax)
    avgErspt = np.zeros(nc)
    for i in range(nc):
        avgErspt[i] = rsptt.averageE(w[mask],p_rspti[i,mask])
    hyb_matrix = np.zeros((nc,nc,len(w[mask])),dtype=np.complex)
    for i in range(nc):
        hyb_matrix[i,i,:] = hyb[i,mask]
    # Interpolate sigma on wd mesh to w mesh
    sig = np.zeros((nc,nc,len(w)),dtype=np.complex)
    for i in range(nc):
        for j in range(nc):
            sig[i,j,:] =  interp1d(wd,sigma[i,j,:],
                                   bounds_error=False,
                                   fill_value=0)(w)
    sig_mask = sig[:,:,mask]
    # Trial energies
    e_start = e0.copy()
    # Optimize epsilon by fitting to interacting PDOS, 
    # while keeping bath parameters fix
    res = minimize(errv, e_start, 
                   args=(avgErspt,w[mask],eim,hyb_matrix,sig_mask),
                   method='SLSQP',
                   bounds=[(wmin,wmax)]*no,
                   options={'maxiter':100,'disp':True})
    e = res.x
    if verbose_text:
        print 'errv(e_start) = ',errv(
            e_start,avgErspt,w[mask],eim,hyb_matrix,sig_mask)
        print 'errv(e) = ',res.fun
        print 'nit = ',res.nit
        print 'success:',res.success
        print 'message:',res.message
    # Calculate approximative PDOS
    if nc == no:
        p = np.diagonal(rsptt.pdos(w,eim,e,hyb,sig)).T
    elif nc == 2*no:
        p = np.diagonal(rsptt.pdos(w,eim,2*list(e),hyb,sig)).T
    print 'e_rspt:',e_rspt,'eV'
    print 'e_0d:',e0d,'eV'
    if off_diag_hyb:
        print 'e_0:',e0,'eV'
    print 'e:',e,'eV'

#### Plot interacting PDOS, using adjusted on-site energies $\epsilon$

In [ ]:
if verbose_fig and self_energy:
    # Plot trace
    plt.figure()
    plt.plot(wd,np.sum(p_rspt,axis=0),label='p_rspt')
    plt.plot(w,np.sum(p,axis=0),label='p')
    plt.legend()
    plt.ylabel('PDOS')
    plt.xlim(xlim)
    plt.show()
    # Plot orbital resolved PDOS
    fig,axes = plt.subplots(nrows=nc,sharex=True,sharey=True)
    for i in range(nc):
        axes[i].plot(w,p_rspti[i,:],label='p_rspt')
        axes[i].plot(w,p[i,:],label='p')
    axes[0].legend()
    axes[-1].set_xlabel('energy  (eV)')
    plt.xlim(xlim)
    plt.subplots_adjust(hspace=0)
    plt.show()

if verbose_fig and self_energy and off_diag_hyb:
    es = [e_rspt,e0d,e0,e]
    labels = ['$\epsilon_\mathrm{rspt}$','$\epsilon_{0,d}$',
              '$\epsilon_0$','$\epsilon$']
    # Plot trace. 
    # Verify different on-site energies by looking at 
    # the interacting PDOS
    plt.figure()
    plt.plot(w,np.sum(p_rspti,axis=0),'k',label='RSPt')
    for en,label in zip(es,labels):
        if nc == no:
            tmp = np.diagonal(rsptt.pdos(w,eim,en,hyb,sig)).T
        elif nc == 2*no:
            tmp = np.diagonal(rsptt.pdos(w,eim,2*list(en),
                                         hyb,sig)).T
        plt.plot(w,np.sum(tmp,axis=0),label=label)
    plt.legend()
    plt.xlabel('energy  (eV)')
    plt.xlim(xlim)
    plt.show()
    # Plot up and down spin
    if spinpol:
        plt.figure()
        plt.plot(wd,-np.sum(p_rspt[:norb,:],axis=0),
                 c='k',label='RSPt')
        plt.plot(wd,np.sum(p_rspt[norb:,:],axis=0),c='k')
        for k,(en,label) in enumerate(zip(es,labels)):
            if nc == no:
                tmp = np.diagonal(rsptt.pdos(w,eim,en,hyb,sig)).T
            elif nc == 2*no:
                tmp = np.diagonal(rsptt.pdos(w,eim,2*list(en),
                                             hyb,sig)).T
            plt.plot(w,-np.sum(tmp[:norb,:],axis=0),c='C'+str(k),
                     label=label)
            plt.plot(w,np.sum(tmp[norb:,:],axis=0),c='C'+str(k))
        plt.legend()
        plt.xlabel('energy  (eV)')
        plt.xlim(xlim)
        plt.show()

### Construct the single-particle Hamiltonian in the Anderson model 
Parameters: $\epsilon, \epsilon_b, V_b$

In [ ]:
if e_method == 0:
    e_onsite = e0d
elif e_method == 1:
    e_onsite = e0
elif e_method == 2:
    e_onsite = e

# Initialize the full Hamiltonian, including spin
h = np.zeros(2*norb*(1+nb)*np.array([1,1]))

# On-site energies of correlated orbitals
if no == 2*norb:
    for i in range(no):
        h[i,i] = e_onsite[i]
elif no == norb:
    for i in range(no):
        h[i,i] = e_onsite[i]
        h[no+i,no+i] = e_onsite[i]

# Bath energies    
if spinpol:
    for i in range(nc):
        for j in range(nb):
            k = nc + i + nc*j
            h[k,k] = eb[i,j]
else: 
    for i in range(nc):
        for j in range(nb):
            k = 2*nc + i + 2*nc*j
            h[k,k] = eb[i,j]
            h[k+no,k+no] = eb[i,j]

# Hoppings
if spinpol:
    for i in range(nc):
        for j in range(nb):
            k = nc + i + nc*j
            h[k,i] = vb[i,j]
            h[i,k] = np.conj(vb[i,j])
else:
    for i in range(nc):
        for j in range(nb):
            k = 2*nc + i + 2*nc*j
            h[k,i] = vb[i,j]
            h[i,k] = np.conj(vb[i,j])
            h[k+no,i+no] = vb[i,j]
            h[i+no,k+no] = np.conj(vb[i,j])
            
if verbose_text:
    print 'Correlated orbitals:'
    print 'Real part:'
    print rsptt.print_matrix(h[:nc,:nc].real)
    print 'Imag part:'
    print rsptt.print_matrix(h[:nc,:nc].imag)
    print 'First set of bath on-site energies:'
    print 'Real part:'
    print rsptt.print_matrix(h[2*norb:2*norb+nc,
                               2*norb:2*norb+nc].real)
    print 'Imag part:'
    print rsptt.print_matrix(h[2*norb:2*norb+nc,
                               2*norb:2*norb+nc].imag)
    

#### Obtain eigenvectors used to rotate from spherical harmonics to transformed basis

In [ ]:
if cubic_basis:
    vtr = unitarytransform.get_spherical_2_cubic_matrix(
        spinpol,(norb-1)/2)
elif irr_flag:
    vtr = np.zeros((nc,nc),dtype=np.complex)
    with open((foldername 
               + 'proj-' 
               + basis_tag 
               + '-' 
               + irr_flag
               + '.inp'),'r') as fileread:
        content = fileread.readlines()
    for row in content[1:]:
        lst = row.split()
        vtr[int(lst[0])-1,int(lst[1])-1] = float(lst[2])+1j*float(lst[3]) 
else:
    # No unitary transformation needed
    # Spherical harmonics to spherical harmonics 
    vtr = np.eye(nc,dtype=np.complex)
    
if verbose_text:
    print "Eigenvectors:"
    print 'Real part:'
    print rsptt.print_matrix(vtr.real)
    print 'Imag part:'
    print rsptt.print_matrix(vtr.imag)

#### Rotate back to spherical harmonics basis

In [ ]:
utr = np.transpose(np.conj(vtr))
u = np.zeros_like(h,dtype=np.complex)
for i in range(2*norb*(1+nb)):
    u[i,i] = 1
if spinpol:
    u[:nc,:nc] = utr
else:
    u[:nc,:nc] = utr
    u[nc:2*nc,nc:2*nc] = utr
h_sph = np.dot(np.transpose(np.conj(u)),np.dot(h,u))

print np.shape(h_sph)
if verbose_text:
    print 'Hamiltonian in spherical harmonics basis:'
    print 'Correlated block:'
    print 'Real part:'
    print rsptt.print_matrix(np.real(h_sph[:2*norb,:2*norb]))   
    print 'Imag part:'
    print rsptt.print_matrix(np.imag(h_sph[:2*norb,:2*norb]))   
    print
    print 'Hopping to first bath:'
    print 'Real part:'
    print rsptt.print_matrix(np.real(h_sph[2*norb:4*norb,:2*norb]))   
    print 'Imag part:'
    print rsptt.print_matrix(np.imag(h_sph[2*norb:4*norb,:2*norb]))   

#### Write Hamiltonian in Quanty format
- In Quanty, the orbital ordering is differently than in RSPt.
A reordering is done.

- Then the Hamiltonian is split up in several parts. 
This is done because the .lua file is not read properly by Quanty is the operator contains "too" many elements.
It seems the column with is somehow limited in .lua files (when used by Quanty).

- Due to the 2p core states, one has to be careful about the indices and shift the indices

In [ ]:
# Convert Hamiltonian to Quanty's orbital ordering format
h_quanty = np.zeros_like(h_sph)
# Number of spin orbitals with a certain spin
ns = norb*(1+nb)
mask_dn = np.full_like(h_quanty,False, dtype=bool)
mask_up = np.full_like(h_quanty,False, dtype=bool)
for i in range(1+nb):
    for j in range(1+nb):
        mask_dn[2*norb*i:(2*i+1)*norb,
                2*norb*j:(2*j+1)*norb] = True
        mask_up[(2*i+1)*norb:2*norb*(i+1),
                (2*j+1)*norb:2*norb*(j+1)] = True
# Down spin
h_quanty[::2,::2] = h_sph[mask_dn].reshape((ns,ns))
# Up spin
h_quanty[1::2,1::2] = h_sph[mask_up].reshape((ns,ns))
# Number of spin-orbitals in the system studied in Quanty    
nf = 6 + 2*norb*(1+nb)
# Index shifts
ishift,jshift = 6,6
# Maximum number of elements in each operator
n = 85
# Transform matrix into several Quanty operators.
# Save to disk.
quantyt.write_quanty_opps(h_quanty,nf,n=n,op_name='H_0',
                          ishift=ishift,jshift=jshift,
                          filename=foldername+'H0.lua')
# Save also all elements into one operator
save21 = False
if save21:
    # Transform matrix into one Quanty operator. 
    # Save to disk.
    quantyt.write_quanty_opp(h_quanty,nf,op_name='H_0',
                             ishift=ishift,jshift=jshift,
                             filename=foldername+'H0.lua')